# PyCity Schools Analysis

* As a whole, schools with higher budgets, did not yield better test results. By contrast, schools with higher spending per student actually (\$645-675) underperformed compared to schools with smaller budgets (<\$585 per student).

* As a whole, smaller and medium sized schools dramatically out-performed large sized schools on passing math performances (89-91% passing vs 67%).

* As a whole, charter schools out-performed the public district schools across all metrics. However, more analysis will be required to glean if the effect is due to school practices or the fact that charter schools tend to serve smaller student populations per school. 
---

### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [16]:
# Dependencies and Setup
import pandas as pd
import numpy as np

# Read School and Student Data File and store into Pandas Data Frames
df_school_raw = pd.read_csv('Resources/schools_complete.csv')
df_student_raw = pd.read_csv('Resources/students_complete.csv')

# Combine the data into a single dataset
df_school_data = pd.merge(df_student_raw, df_school_raw, how="left", left_on='school', right_on='name')

# Rename, remove, and sort columns in merged dataset
df_school_data = df_school_data.rename(columns={'Student ID':'student_id',
                                                'name_x':'student_name',
                                                'School ID':'school_id'
                                               })

df_school_data = df_school_data[['student_id','student_name','gender','grade',
                                 'reading_score','math_score','school_id','school',
                                 'type','size','budget'
                                ]]

df_school_data = df_school_data.set_index('student_id')
df_school_data.head()


,student_name,gender,grade,reading_score,math_score,school_id,school,type,size,budget
student_id,,,,,,,,,,
0,Paul Bradley,M,9th,66,79,0,Huang High School,District,2917,1910635
1,Victor Smith,M,12th,94,61,0,Huang High School,District,2917,1910635
2,Kevin Rodriguez,M,12th,90,60,0,Huang High School,District,2917,1910635
3,Dr. Richard Scott,M,12th,67,58,0,Huang High School,District,2917,1910635
4,Bonnie Ray,F,9th,97,84,0,Huang High School,District,2917,1910635


## District Summary

* Calculate the total number of schools

* Calculate the total number of students

* Calculate the total budget

* Calculate the average math score 

* Calculate the average reading score

* Calculate the overall passing rate (overall average score), i.e. (avg. math score + avg. reading score)/2

* Calculate the percentage of students with a passing math score (70 or greater)

* Calculate the percentage of students with a passing reading score (70 or greater)

* Create a dataframe to hold the above results

* Optional: give the displayed data cleaner formatting

In [17]:
#Create and store District Summary
total_schools = len(df_school_data['school'].unique())
total_students = len(df_school_data['student_name'])
total_budget = df_school_raw['budget'].sum()
dist_avg_math_score = df_school_data['math_score'].mean()
dist_avg_reading_score = df_school_data['reading_score'].mean()
dist_overall_score = (dist_avg_math_score + dist_avg_reading_score)/2
dist_pass_math_pct = len(df_school_data[df_school_data.math_score >= 70])/total_students
dist_pass_read_pct = len(df_school_data[df_school_data.reading_score >= 70])/total_students

df_dist_summary = pd.DataFrame([{0:total_schools,
                                 1:total_students,
                                 2:total_budget,
                                 3:dist_avg_math_score,
                                 4:dist_avg_reading_score,
                                 5:dist_overall_score,
                                 6:dist_pass_math_pct,
                                 7:dist_pass_read_pct
                                }])

#Reformat District Summary dataframe
df_dist_summary[0] = df_dist_summary[0].map('{:,}'.format)
df_dist_summary[1] = df_dist_summary[1].map('{:,}'.format)
df_dist_summary[2] = df_dist_summary[2].map('${:,}'.format)
df_dist_summary[3] = df_dist_summary[3].map('{:.2f}'.format)
df_dist_summary[4] = df_dist_summary[4].map('{:.2f}'.format)
df_dist_summary[5] = df_dist_summary[5].map('{:.2f}'.format)
df_dist_summary[6] = df_dist_summary[6].map('{:.2%}'.format)
df_dist_summary[7] = df_dist_summary[7].map('{:.2%}'.format)
                                 
df_dist_summary.columns = ['Total Schools',
                           'Total Students',
                           'Total Budget',
                           'Average Math Score',
                           'Average Reading Score',
                           'Average Overall Score',
                           '% Passing Math',
                           '% Passing Reading'
                          ]

df_dist_summary

,Total Schools,Total Students,Total Budget,Average Math Score,Average Reading Score,Average Overall Score,% Passing Math,% Passing Reading
0,15,"39,170","$24,649,428",78.99,81.88,80.43,74.98%,85.81%


## School Summary

* Create an overview table that summarizes key metrics about each school, including:
  * School Name
  * School Type
  * Total Students
  * Total School Budget
  * Per Student Budget
  * Average Math Score
  * Average Reading Score
  * % Passing Math
  * % Passing Reading
  * Overall Passing Rate (Average of the above two)
  
* Create a dataframe to hold the above results

In [18]:
#Create new dataframe of school data grouped by school ID
df_school_group = df_school_data.groupby('school')

df_school_summary = pd.DataFrame()
df_school_summary['School Type'] = df_school_group.type.unique().str[0]
df_school_summary['Total Students'] = df_school_group.student_name.count()
df_school_summary['Total School Budget'] = df_school_group.budget.unique().str[0]
df_school_summary['Per Student Budget'] = df_school_summary['Total School Budget']/df_school_summary['Total Students']
df_school_summary['Average Math Score'] = df_school_group.math_score.mean()
df_school_summary['Average Reading Score'] = df_school_group.reading_score.mean()

#Create new dataframes for calculate passing data by school
df_passing_math = df_school_data.loc[df_school_data.math_score >= 70,['math_score','school']].groupby('school')
df_passing_reading = df_school_data.loc[df_school_data.reading_score >= 70,['reading_score','school']].groupby('school')

df_school_summary['% Passing Math'] = df_passing_math.math_score.count()/df_school_summary['Total Students']
df_school_summary['% Passing Reading'] = df_passing_reading.reading_score.count()/df_school_summary['Total Students']
df_school_summary['% Overall Passing'] = (df_school_summary['% Passing Math'] + df_school_summary['% Passing Reading'])/2


#Reformat School Summary Table
df_school_summary['Total Students'] = df_school_summary['Total Students'].map('{:,}'.format)
df_school_summary['Total School Budget'] = df_school_summary['Total School Budget'].map('${:,}'.format)
df_school_summary['Per Student Budget'] = df_school_summary['Per Student Budget'].map('${:,.0f}'.format)
df_school_summary['Average Math Score'] = df_school_summary['Average Math Score'].map('{:.2f}'.format)
df_school_summary['Average Reading Score'] = df_school_summary['Average Reading Score'].map('{:.2f}'.format)
df_school_summary['% Passing Math'] = df_school_summary['% Passing Math'].map('{:.2%}'.format)
df_school_summary['% Passing Reading'] = df_school_summary['% Passing Reading'].map('{:.2%}'.format)
df_school_summary['% Overall Passing'] = df_school_summary['% Overall Passing'].map('{:.2%}'.format)

del df_school_summary.index.name
df_school_summary


,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
Bailey High School,District,"4,976","$3,124,928",$628,77.05,81.03,66.68%,81.93%,74.31%
Cabrera High School,Charter,"1,858","$1,081,356",$582,83.06,83.98,94.13%,97.04%,95.59%
Figueroa High School,District,"2,949","$1,884,411",$639,76.71,81.16,65.99%,80.74%,73.36%
Ford High School,District,"2,739","$1,763,916",$644,77.10,80.75,68.31%,79.30%,73.80%
Griffin High School,Charter,"1,468","$917,500",$625,83.35,83.82,93.39%,97.14%,95.27%
Hernandez High School,District,"4,635","$3,022,020",$652,77.29,80.93,66.75%,80.86%,73.81%
Holden High School,Charter,427,"$248,087",$581,83.80,83.81,92.51%,96.25%,94.38%
Huang High School,District,"2,917","$1,910,635",$655,76.63,81.18,65.68%,81.32%,73.50%
Johnson High School,District,"4,761","$3,094,650",$650,77.07,80.97,66.06%,81.22%,73.64%
Pena High School,Charter,962,"$585,858",$609,83.84,84.04,94.59%,95.95%,95.27%


## Top Performing Schools (By Passing Rate)

* Sort and display the top five schools in overall passing rate

In [19]:
#Top Performing schools (by passing rate)
df_school_summary = df_school_summary.sort_values('% Overall Passing',ascending=False)
df_school_summary.head()


,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
Cabrera High School,Charter,"1,858","$1,081,356",$582,83.06,83.98,94.13%,97.04%,95.59%
Thomas High School,Charter,"1,635","$1,043,130",$638,83.42,83.85,93.27%,97.31%,95.29%
Griffin High School,Charter,"1,468","$917,500",$625,83.35,83.82,93.39%,97.14%,95.27%
Pena High School,Charter,962,"$585,858",$609,83.84,84.04,94.59%,95.95%,95.27%
Wilson High School,Charter,"2,283","$1,319,574",$578,83.27,83.99,93.87%,96.54%,95.20%


## Bottom Performing Schools (By Passing Rate)

* Sort and display the five worst-performing schools

In [20]:
#Bottom Performing schools (by passing rate)
df_school_summary = df_school_summary.sort_values('% Overall Passing')
df_school_summary.head()

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
Rodriguez High School,District,"3,999","$2,547,363",$637,76.84,80.74,66.37%,80.22%,73.29%
Figueroa High School,District,"2,949","$1,884,411",$639,76.71,81.16,65.99%,80.74%,73.36%
Huang High School,District,"2,917","$1,910,635",$655,76.63,81.18,65.68%,81.32%,73.50%
Johnson High School,District,"4,761","$3,094,650",$650,77.07,80.97,66.06%,81.22%,73.64%
Ford High School,District,"2,739","$1,763,916",$644,77.10,80.75,68.31%,79.30%,73.80%


## Math Scores by Grade

* Create a table that lists the average Reading Score for students of each grade level (9th, 10th, 11th, 12th) at each school.

  * Create a pandas series for each grade. Hint: use a conditional statement.
  
  * Group each series by school
  
  * Combine the series into a dataframe
  
  * Optional: give the displayed data cleaner formatting

In [21]:
#Create dataframes for each grade
df_9th_scores = df_school_data.loc[df_school_data.grade == '9th', ['school','math_score','reading_score']].groupby('school')
df_10th_scores = df_school_data.loc[df_school_data.grade == '10th', ['school','math_score','reading_score']].groupby('school')
df_11th_scores = df_school_data.loc[df_school_data.grade == '11th', ['school','math_score','reading_score']].groupby('school')
df_12th_scores = df_school_data.loc[df_school_data.grade == '12th', ['school','math_score','reading_score']].groupby('school')

#Create math score summary table
df_grade_math_scores = pd.DataFrame()
df_grade_math_scores['9th'] = df_9th_scores.math_score.mean().map('{:.2f}'.format)
df_grade_math_scores['10th'] = df_10th_scores.math_score.mean().map('{:.2f}'.format)
df_grade_math_scores['11th'] = df_11th_scores.math_score.mean().map('{:.2f}'.format)
df_grade_math_scores['12th'] = df_12th_scores.math_score.mean().map('{:.2f}'.format)

del df_grade_math_scores.index.name
df_grade_math_scores

,9th,10th,11th,12th
Bailey High School,77.08,77.00,77.52,76.49
Cabrera High School,83.09,83.15,82.77,83.28
Figueroa High School,76.40,76.54,76.88,77.15
Ford High School,77.36,77.67,76.92,76.18
Griffin High School,82.04,84.23,83.84,83.36
Hernandez High School,77.44,77.34,77.14,77.19
Holden High School,83.79,83.43,85.00,82.86
Huang High School,77.03,75.91,76.45,77.23
Johnson High School,77.19,76.69,77.49,76.86
Pena High School,83.63,83.37,84.33,84.12


## Reading Score by Grade 

* Perform the same operations as above for reading scores

In [22]:
#Create reading score summary table
df_grade_reading_scores = pd.DataFrame()
df_grade_reading_scores['9th'] = df_9th_scores.reading_score.mean().map('{:.2f}'.format)
df_grade_reading_scores['10th'] = df_10th_scores.reading_score.mean().map('{:.2f}'.format)
df_grade_reading_scores['11th'] = df_11th_scores.reading_score.mean().map('{:.2f}'.format)
df_grade_reading_scores['12th'] = df_12th_scores.reading_score.mean().map('{:.2f}'.format)

del df_grade_reading_scores.index.name
df_grade_reading_scores

,9th,10th,11th,12th
Bailey High School,81.30,80.91,80.95,80.91
Cabrera High School,83.68,84.25,83.79,84.29
Figueroa High School,81.20,81.41,80.64,81.38
Ford High School,80.63,81.26,80.40,80.66
Griffin High School,83.37,83.71,84.29,84.01
Hernandez High School,80.87,80.66,81.40,80.86
Holden High School,83.68,83.32,83.82,84.70
Huang High School,81.29,81.51,81.42,80.31
Johnson High School,81.26,80.77,80.62,81.23
Pena High School,83.81,83.61,84.34,84.59


## Scores by School Spending

* Create a table that breaks down school performances based on average Spending Ranges (Per Student). Use 4 reasonable bins to group school spending. Include in the table each of the following:
  * Average Math Score
  * Average Reading Score
  * % Passing Math
  * % Passing Reading
  * Overall Passing Rate (Average of the above two)

In [23]:
# Sample bins. Feel free to create your own bins.
spending_bins = [0, 585, 615, 645, 675]
group_names = ["<$585", "$585-615", "$615-645", "$645-675"]

In [24]:
#Create dataframe for school spending
df_school_spend = df_school_summary[['Average Math Score',
                                        'Average Reading Score',
                                        '% Passing Math',
                                        '% Passing Reading',
                                        '% Overall Passing',
                                        'Per Student Budget'
                                       ]]

#Remove number formatting from dataframe
df_school_spend = df_school_spend.replace('\$','',regex=True)
df_school_spend = df_school_spend.replace('%','',regex=True)
df_school_spend = df_school_spend.replace(',','',regex=True)
df_school_spend = df_school_spend.astype(np.float)

#Add spending ranges to table
df_school_spend['Spending Ranges (Per Student)'] = pd.cut(df_school_spend['Per Student Budget'],spending_bins,labels=group_names)
df_school_spend = df_school_spend.groupby('Spending Ranges (Per Student)')

#Create formatted table
df_scores_by_spending = pd.DataFrame()
df_scores_by_spending['Average Math Score'] = df_school_spend['Average Math Score'].mean().map('{:.2f}'.format)
df_scores_by_spending['Average Reading Score'] = df_school_spend['Average Reading Score'].mean().map('{:.2f}'.format)
df_scores_by_spending['% Passing Math'] = df_school_spend['% Passing Math'].mean().map('{:.2f}%'.format)
df_scores_by_spending['% Passing Reading'] = df_school_spend['% Passing Reading'].mean().map('{:.2f}%'.format)
df_scores_by_spending['% Overall Passing'] = df_school_spend['% Overall Passing'].mean().map('{:.2f}%'.format)

df_scores_by_spending


,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
Spending Ranges (Per Student),,,,,
<$585,83.45,83.93,93.46%,96.61%,95.03%
$585-615,83.60,83.89,94.23%,95.90%,95.06%
$615-645,79.08,81.89,75.67%,86.11%,80.89%
$645-675,77.00,81.03,66.16%,81.13%,73.65%


## Scores by School Size

* Perform the same operations as above, based on school size.

In [25]:
# Sample bins. Feel free to create your own bins.
size_bins = [0, 1000, 2000, 5000]
size_group_names = ["Small (<1000)", "Medium (1000-2000)", "Large (2000-5000)"]

In [26]:
#Create dataframe for school size
df_school_size = df_school_summary[['Average Math Score',
                                        'Average Reading Score',
                                        '% Passing Math',
                                        '% Passing Reading',
                                        '% Overall Passing',
                                        'Total Students'
                                       ]]

#Remove number formatting from dataframe
df_school_size = df_school_size.replace('\$','',regex=True)
df_school_size = df_school_size.replace('%','',regex=True)
df_school_size = df_school_size.replace(',','',regex=True)
df_school_size = df_school_size.astype(np.float)

#Add school size ranges to table
df_school_size['School Size'] = pd.cut(df_school_size['Total Students'],size_bins,labels=size_group_names)
df_school_size = df_school_size.groupby('School Size')

#Create formatted table
df_scores_by_size = pd.DataFrame()
df_scores_by_size['Average Math Score'] = df_school_size['Average Math Score'].mean().map('{:.2f}'.format)
df_scores_by_size['Average Reading Score'] = df_school_size['Average Reading Score'].mean().map('{:.2f}'.format)
df_scores_by_size['% Passing Math'] = df_school_size['% Passing Math'].mean().map('{:.2f}%'.format)
df_scores_by_size['% Passing Reading'] = df_school_size['% Passing Reading'].mean().map('{:.2f}%'.format)
df_scores_by_size['% Overall Passing'] = df_school_size['% Overall Passing'].mean().map('{:.2f}%'.format)

df_scores_by_size

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
School Size,,,,,
Small (<1000),83.82,83.93,93.55%,96.10%,94.82%
Medium (1000-2000),83.37,83.87,93.60%,96.79%,95.20%
Large (2000-5000),77.75,81.34,69.96%,82.77%,76.36%


## Scores by School Type

* Perform the same operations as above, based on school type.

In [27]:
#Create dataframe for school type
df_school_type = df_school_summary[['Average Math Score',
                                        'Average Reading Score',
                                        '% Passing Math',
                                        '% Passing Reading',
                                        '% Overall Passing',
                                       ]]

#Remove number formatting from dataframe
df_school_type = df_school_type.replace('\$','',regex=True)
df_school_type = df_school_type.replace('%','',regex=True)
df_school_type = df_school_type.replace(',','',regex=True)
df_school_type = df_school_type.astype(np.float)

#Add and group by school type column 
df_school_type['School Type'] = df_school_summary['School Type']
df_school_type = df_school_type.groupby('School Type')

#Create formatted table
df_scores_by_type = pd.DataFrame()
df_scores_by_type['Average Math Score'] = df_school_type['Average Math Score'].mean().map('{:.2f}'.format)
df_scores_by_type['Average Reading Score'] = df_school_type['Average Reading Score'].mean().map('{:.2f}'.format)
df_scores_by_type['% Passing Math'] = df_school_type['% Passing Math'].mean().map('{:.2f}%'.format)
df_scores_by_type['% Passing Reading'] = df_school_type['% Passing Reading'].mean().map('{:.2f}%'.format)
df_scores_by_type['% Overall Passing'] = df_school_type['% Overall Passing'].mean().map('{:.2f}%'.format)

df_scores_by_type

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
School Type,,,,,
Charter,83.47,83.90,93.62%,96.59%,95.10%
District,76.96,80.97,66.55%,80.80%,73.67%
